In [ ]:
import sys # biblioteca para controlar a execução do programa
import vlc # Importa o VLC media player, necessário para a execução do video
import os # Módulo de manipulação do sistema operacional, permitindo a interação com o OS

# PyQt5.QtWidgets contém os elemenos da interface gráfica, como botões, janelas e sliders
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QFileDialog, QSlider, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QShortcut, QFrame
# Biblioteca necessárias para configuração e atualização da barra de rolagem
from PyQt5.QtCore import Qt, QTimer

from PyQt5.QtGui import QKeySequence, QFont

In [ ]:
# Classe responsável pela criação de uma janela específica para os controles do player
class VideoPlayer(QMainWindow):
    
    # Construtor da classe
    def __init__(self):

        super().__init__() # realiza a chamada dos métodos da classe pai (análogo à classe VideoWindow())

        self.setWindowTitle("Video Player com VLC") # Define o título da janela do player
        self.setGeometry(100, 100, 900, 500)  # Define a geometria da janela 

        # Criando um widget central, ele será responsável por aagrupar o conteúdo da janela
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        # Define o layout principal, pode ser entendido
        # como uma div do html. neste caso o conteúdo vai
        # ser exibido lado-a-lado horizontalmente
        self.main_layout = QHBoxLayout(self.central_widget)

        # Criar um Frame para o Player VLC, aqui será 
        # renderizado o vídeo
        self.video_frame = QFrame(self) # Instância do frame
        self.video_frame.setFixedSize(800, 800) # Ajuste fixo do tamanho do vídeo
        self.main_layout.addWidget(self.video_frame) # Adiciona o frame ao widget central

        # Define um layout vertical a fim de agrupar os botões de controle
        # verticalmente, um abaixo do outro
        self.control_layout = QVBoxLayout()

        # Adiciona o layout de controle ao layout principal
        # podemos entender este comando como a adição de uma div dentro de outra
        self.main_layout.addLayout(self.control_layout) 

        # Layout horizontal para alguns do botões de controle
        # o qual sera adicionado posteriormente ao control_layout
        # seguindo a ordem de disposição dos botões
        self.hor_layout = QHBoxLayout()
        self.hor_layout2 = QHBoxLayout()

        #-------------------------------------------------------------------------------------------------------------

        # Criação da Janela do Vídeo, sendo ela uma intância 
        # da classe VideoWindow() criada préviamente, além de 
        # associar o VLC à janela criada.

        self.instance = vlc.Instance() # cria uma instância do framework do VLC
        self.media_player = self.instance.media_player_new() # cria um novo player de vídeo a partir da instância do vlc
        self.media_player.set_hwnd(int(self.video_frame.winId())) # Vincula o player do VLC à janela criada

        #-------------------------------------------------------------------------------------------------------------

        # Definição elementos de controle (botões e slider)

        self.slider = QSlider(Qt.Horizontal) # slider
        self.open_button = QPushButton("Abrir Vídeo") # botão para abrir vídeo
        self.next_frame_btn = QPushButton("Avançar frame") # botão para Avançar frame
        self.prev_frame_btn = QPushButton("Retroceder frame") # botão para Retroceder frame
        self.play_button = QPushButton("Play/Pause") # botão para dar play ou pause
        self.exit_button = QPushButton("Fechar programa") # botão para sair do programa
        self.save_ind = QPushButton("Salvar frame como 'Indolor'")  # botão responsável por salvar o frame na pasta 'indolor'
        self.save_pd = QPushButton("Salvar frame como 'Pouca dor'") # botão responsável por salvar o frame na pasta 'pouca dor'
        self.save_md = QPushButton("Salvar frame como 'Muita dor'") # botão responsável por salvar o frame na pasta 'muita dor'
        self.change_keys = QPushButton("Escolher novas teclas de atalho") # botão responsável por modificar as teclas de atalho

        # Definindo slider para o ajuste da velocidade do vídeo

        self.speed_slider = QSlider(Qt.Horizontal) # slider para a velocidade de reprodução do vídeo
        self.speed_label = QLabel("Velocidade: 1x")  # Exibe a velocidade atual como uma label

        # Configurando slider de velocidade
        self.speed_slider.setMinimum(0)  # Definição do menor índice 0 = 1x
        self.speed_slider.setMaximum(4)  # Definição do maior índice 4 = 16x
        self.speed_slider.setTickInterval(1)  # Definindo a taxa de incremento de 1 em 1
        self.speed_slider.setTickPosition(QSlider.TicksBelow) # Define a posição do ponteiro do slider
        self.speed_slider.setValue(0)  # Definindo o valor (posição) inicial do slider em 1x

        #-------------------------------------------------------------------------------------------------------------

        # Inserindo os elementos criados na janela de controles
        # observação: A ordem importa para a visualização

        self.control_layout.addWidget(self.play_button)

        # O layout horizontal deve ser inserido em ordem junto com seus widgets
        self.control_layout.addLayout(self.hor_layout)
        self.hor_layout.addWidget(self.next_frame_btn)
        self.hor_layout.addWidget(self.prev_frame_btn)

        self.time_label = QLabel("00:00 / 00:00") # Instância de um label (formato "00:00 / 00:00")
        self.control_layout.addWidget(self.time_label) # Inserção do label como elemento da janela de controles
        self.control_layout.addWidget(self.slider)

        self.speed_label = QLabel("Velocidade: 1x")  # Exibe a velocidade atual
        self.control_layout.addWidget(self.speed_label)
        self.control_layout.addWidget(self.speed_slider)

        # O layout horizontal deve ser inserido em ordem junto com seus widgets
        self.control_layout.addLayout(self.hor_layout2)
        self.hor_layout2.addWidget(self.save_ind)
        self.hor_layout2.addWidget(self.save_pd)
        self.hor_layout2.addWidget(self.save_md)

        self.control_layout.addWidget(self.open_button)

        # Inserção de labels para as teclas de atalho

        # Variáveis de tecla de atalho

        openFile = "o"
        playPause = "Space"
        nextFrame = "d"
        prevFrame = "a"
        exitProg = "q"
        saveIND = "j"
        saveMD = "l"
        savePD = "k"

        self.keys_label = QLabel()
        font = QFont("Arial", 10)  # Nome, tamanho, peso (opcional)
        self.keys_label.setFont(font)
        
        # Define a label como rich text para permitir a inclusão de comandos HTML
        self.keys_label.setTextFormat(Qt.RichText)

        self.keys_label.setText(
            "<br>"
            "Teclas de atalho para os controles<br>"
            "<br>"
            f"Abrir um arquivo de vídeo: <b>{openFile}</b><br>"
            f"Play/Pause: <b>{playPause}</b><br>"
            f"Avançar frame: <b>{nextFrame}</b><br>"
            f"Retroceder frame: <b>{prevFrame}</b><br>"
            f"Sair do programa: <b>{exitProg}</b><br>"
            f"Salvar frame como 'indolor': <b>{saveIND}</b><br>"
            f"Salvar frame como 'Pouca dor': <b>{savePD}</b><br>"
            f"Salvar frame como 'Muita dor': <b>{saveMD}</b><br>"
            "<br>"
        )
        self.control_layout.addWidget(self.keys_label)

        self.control_layout.addWidget(self.change_keys)
        self.control_layout.addWidget(self.exit_button)

        #-------------------------------------------------------------------------------------------------------------

        # Associa as funções de controle aos botões criados 

        self.open_button.clicked.connect(self.open_file) 
        
        self.play_button.clicked.connect(self.toggle_play_pause) 
        
        self.slider.sliderMoved.connect(self.set_position) 
        
        self.exit_button.clicked.connect(self.exit_program) 
        
        self.next_frame_btn.clicked.connect(self.next_frame)
        
        self.prev_frame_btn.clicked.connect(self.prev_frame)
        
        self.speed_slider.valueChanged.connect(self.change_speed)

        # Em Python, uma expressão lambda é uma função anônima de uma única linha.
        # Ela é usada para definir funções curtas sem precisar usar def.
        # A sintaxe base para uma função lambda é "lambda argumentos: expressão"

        # No caso do trecho a baico cada botão precisa chamar a função capture_frame() com um nome de pasta diferente.
        # Contudo o connect() do PyQt exige uma função sem argumentos, por isso, O lambda cria uma função anônima 
        # temporária que será executada somente quando o botão for clicado.

        # Podemos dizer que lambda: self.capture_frame("Indolor") seria o mesmo que:
        # def anonimous():
        #   self.capture_frame("indolor")

        self.save_ind.clicked.connect(lambda: self.capture_frame("Indolor"))
        self.save_pd.clicked.connect(lambda: self.capture_frame("Pouca dor"))
        self.save_md.clicked.connect(lambda: self.capture_frame("Muita dor"))

        # Timer responsável por atualizar a barra de progresso

        self.timer = QTimer(self) # Cria uma instância do temporizador
        self.timer.setInterval(500) # Define um intervalo de tempo de 500ms
        self.timer.timeout.connect(self.update_slider) # O slider é atualizado a cada 500ms, utilizando a fução update

        #-------------------------------------------------------------------------------------------------------------

        # Associando teclas de atalho a cada uma das funções dos botões

        self.open_button_shortcut = QShortcut(QKeySequence(openFile), self)
        self.open_button_shortcut.activated.connect(self.open_file)

        self.play_button_shortcut = QShortcut(QKeySequence(playPause), self)
        self.play_button_shortcut.activated.connect(self.toggle_play_pause)

        self.next_frame_shortcut = QShortcut(QKeySequence(nextFrame), self)
        self.next_frame_shortcut.activated.connect(self.next_frame)

        self.prev_frame_shortcut = QShortcut(QKeySequence(prevFrame), self)
        self.prev_frame_shortcut.activated.connect(self.prev_frame)

        self.exit_shortcut = QShortcut(QKeySequence(exitProg), self)
        self.exit_shortcut.activated.connect(self.exit_program)

        self.save_ind_shortcut = QShortcut(QKeySequence(saveIND), self)
        self.save_ind_shortcut.activated.connect(lambda: self.capture_frame("Indolor"))

        self.save_pd_shortcut = QShortcut(QKeySequence(savePD), self)
        self.save_pd_shortcut.activated.connect(lambda: self.capture_frame("Pouca dor"))

        self.save_md_shortcut = QShortcut(QKeySequence(saveMD), self)
        self.save_md_shortcut.activated.connect(lambda: self.capture_frame("Muita dor"))
    

#-------------------------------------------------------------------------------------------------------------

    # Captura o frame atual e salva no diretório definido pelo parâmetro "folder_name"

    def capture_frame(self, folder_name):
            
        os.makedirs(folder_name, exist_ok=True) # cria o diretório para armazenar os frames (caso não exista previamente)

        # Pausa o vídeo, caso ele já não esteja pausado 
        if self.media_player.is_playing():

            self.media_player.pause()

        # O VLC não tem uma função pronta para retornar o numero do frame atual, contudo é possível obter este valor
        # por meio do valor de tempo do frame atual e pela taxa de quadros do vídeo (FPS), com isso temos a equação:
            
            # frame atual = tempo atual (ms)/1000 * taxa de quadros

        current_Time = self.media_player.get_time() # tempo do frame atual
        fps = self.media_player.get_fps() # taxa de quadros do vídeo

        if fps > 0:
            frame_number = int((current_Time/1000) * fps)
        else:
            frame_number = 'unknown'
        
        frame_path = os.path.join(folder_name, f"frame_{frame_number}.png") # Organiza o caminho da imagem salva
        self.media_player.video_take_snapshot(0, frame_path, 0, 0) # salva o snapshot do vídeo

        # Observação:
        # video_take_snapshot(num, file_path, width, height)
        # num -> Índice da janela de vídeo (sempre 0 se houver apenas um vídeo).
        # file_path -> Caminho onde a imagem será salva
        # width -> Largura da imagem. Se 0, mantém o tamanho original do vídeo.
        # height -> Altura da imagem. Se 0, mantém o tamanho original do vídeo.

#-------------------------------------------------------------------------------------------------------------

    # Altera a velocidade do vídeo com base no slider
    def change_speed(self):

        self.speed_values = [1.0, 2.0, 4.0, 8.0, 16.0] # Lista de valores de reprodução

        index = self.speed_slider.value()  # Obtém o índice da velocidade
        speed = self.speed_values[index]  # Obtém a taxa correspondente
        self.media_player.set_rate(speed)  # Define a nova taxa de reprodução
        self.speed_label.setText(f"Velocidade: {int(speed)}x")  # Atualiza o rótulo

#-------------------------------------------------------------------------------------------------------------

    # Função responsável por avançar individualmente um frame
    def next_frame(self):

        # Pausa o vídeo, caso ele já não esteja pausado 
        if self.media_player.is_playing():
            self.media_player.pause()

        # Utiliza um comando do próprio vlc para avançar o frame
        self.media_player.next_frame()

     # Função responsável por retroceder individualmente um frame
     # OBS: o vlc não possui uma função pronta para retroceder, por isso, é necessário
     # retroceder manualmente retrocedendo o tempo do vídeo referente à um frame

#-------------------------------------------------------------------------------------------------------------

    def prev_frame(self):

        # Pausa o vídeo, caso ele já não esteja pausado 
        if self.media_player.is_playing():
            self.media_player.pause()
        
        fps = self.media_player.get_fps()  # Obtém FPS do vídeo
        if fps > 0:
            frame_time = int(1000 / fps)  # Tempo de um frame em ms
            current_time = self.media_player.get_time()  # Tempo atual em ms
            new_time = max(0, current_time - frame_time)  # Garante que não vá abaixo de 0
            self.media_player.set_time(new_time)  # Define o novo tempo

#-------------------------------------------------------------------------------------------------------------

    def exit_program(self):

        #não funciona
        #resolver futuramente
        return 0

#-------------------------------------------------------------------------------------------------------------

    # Função responsável por permitir a escolha de um arquivo utilizando o explorer 
    def open_file(self):

        # QFileDialog.getOpenFileName() abre uma janela de seleção de aquivos, retornando dois valores:
        # 1 - O caminho do arquivo selecionado (exemplo: "C:/Videos/filme.mp4").
        # 2 - Um valor extra que contém o filtro de arquivos aplicado

        # Parâmetros utilizados
        # self - Referência à janela principal
        # "Abrir Vídeo" - Título da janela de seleção de arquivos.
        # "" - Diretório inicial (se vazio, abre no último local acessado).
        # "Arquivos de Vídeo (*.mp4 *.avi *.mkv)" - Filtro para exibir apenas arquivos de vídeo.

        file_name, _ = QFileDialog.getOpenFileName(self, "Abrir Vídeo", "", "Arquivos de Vídeo (*.mp4 *.avi *.mkv)")
        
        # Valida se existe um arquivo de vídeo selecionado

        if file_name:

            media = self.instance.media_new(file_name) # Cria um objeto de mídia a parir da instância do  VLC e associa o vídeo selecionado
            self.media_player.set_media(media) # O objeto de mídia (media) é atribuído ao player de vídeo criado pela instância do VLC
            self.media_player.play() # O vlc inicia a reprodução do vídeo carregado
            self.timer.start() # Inicia o timer da barra de progresso

#-------------------------------------------------------------------------------------------------------------

    def toggle_play_pause(self):

        # Valida se o vídeo está reproduzindo
        # Em caso positivo, o vídeo é pausado. Em caso negativo o vídeo é reproduzido

        if self.media_player.is_playing():
            self.media_player.pause()

        else:
            self.media_player.play()

#-------------------------------------------------------------------------------------------------------------
    
    # Define a posição do vídeo ao arrastar o slider.
    def set_position(self, value):

        if self.media_player is not None:

            # observação: value está vindo diretamente do QSlider sempre que o usuário interage com ele.
            # Quando o usuário move o slider, o Qt emite sliderMoved(int) automaticamente, passando o valor do slider para set_position.
            # O connect() pode se conectar a funções que aceitam os argumentos esperados pelo sinal.
            # No caso do sliderMoved, ele emite um único inteiro (o valor do slider), e a função 
            # set_position(self, value) está esperando exatamente um argumento além de self.

            duration = self.media_player.get_length()  # Obtém a duração total do vídeo
            new_time = int((value / self.slider.maximum()) * duration) # Obtem a duração do vídeo de acordo com a posição do slider
            self.media_player.set_time(new_time) # Define o tempo de vídeo de acordo com o tempo obtido pela posição do slider
    
    # Atualiza a posição do slider com base no progresso do vídeo.
    def update_slider(self):

        if self.media_player is not None:

            duration = self.media_player.get_length()  # Duração total do vídeo em ms
            current_time = self.media_player.get_time()  # Tempo atual do vídeo em ms

            if duration > 0:  # Garante que a duração do vídeo é válida
                
                # Define um novo valor para o slider, levando em consideração o tempo de vídeo e o limite do slider
                slider_value = int((current_time / duration) * self.slider.maximum())
                self.slider.setValue(slider_value)

            # Atualiza o rótulo do tempo na interface
            current_sec = current_time // 1000
            duration_sec = duration // 1000
            self.time_label.setText(f"{current_sec // 60:02}:{current_sec % 60:02} / {duration_sec // 60:02}:{duration_sec % 60:02}")

    
#-------------------------------------------------------------------------------------------------------------
    
    # Formatação do tempo como (mm:ss)
    def format_time(self, seconds):

        minutes = seconds // 60
        seconds = seconds % 60
        return f"{minutes:02}:{seconds:02}"

In [3]:
app = QApplication(sys.argv)
player = VideoPlayer()
player.show()
sys.exit(app.exec())

SystemExit: 0

C:\Users\marci_wawp\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
